In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt

import pandas as pd
import csv
import numpy as np
import datetime
import re
import warnings
from IPython.display import HTML, display
import requests
import sys
from scipy.optimize import curve_fit

warnings.filterwarnings('ignore')
plt.ioff()

%config InlineBackend.figure_format = 'svg'

df, obs_pred_r2_G, model_G, loc, obs_x_G, obs_y_G, pred_y_G, forecasted_x_G, forecasted_y_G = [],[],[],[],[],[],[],[],[]
dates, fdates = [],[]

In [2]:

def logistic(x, a, b, c, d):
    return a / (d + np.exp(-c * x + b))

def exponential(x, a, b, c, d):
    return a * (d + (np.exp(b * x) + c))

def obs_pred_rsquare(obs, pred):
    return 1 - sum((obs - pred) ** 2) / sum((obs - np.mean(obs)) ** 2)


def get_logistic(obs_x, obs_y):

    obs_x = np.array(obs_x)
    for i, val in enumerate(obs_y):
        if val == 0:
            try:
                obs_y[i] = obs_y[i-1]
            except:
                pass       
    
    obs_y = np.array(obs_y)
    
    try:
        popt, pcov = curve_fit(logistic, obs_x, obs_y)
        pred_y = logistic(obs_x, *popt)
        forecasted_x = np.array(list(range(max(obs_x) + 7)))
        forecasted_y = logistic(forecasted_x, *popt)
        
    except:
        print('Logistic failed to fit. Using 3rd degree polynomial.')
        forecasted_y, forecasted_x, pred_y = get_polynomial(obs_x, obs_y)
        
    return forecasted_y, forecasted_x, pred_y



def get_exponential(obs_x, obs_y):
    
    obs_x = np.array(obs_x)
    
    for i, val in enumerate(obs_y):
        if val == 0:
            try:
                obs_y[i] = obs_y[i-1]
            except:
                pass       
    
    obs_y = np.array(obs_y)
    
    try:
        popt, pcov = curve_fit(exponential, obs_x, obs_y)
        pred_y = exponential(obs_x, *popt)
        forecasted_x = np.array(list(range(max(obs_x) + 7)))
        forecasted_y = exponential(forecasted_x, *popt)
        
    except:
        print('Exponential failed to fit. Using 3rd degree polynomial.')
        forecasted_y, forecasted_x, pred_y = get_polynomial(obs_x, obs_y)
        
    return forecasted_y, forecasted_x, pred_y
        


def get_polynomial(obs_x, obs_y, degree=9):
    
    obs_x = np.array(obs_x)
    for i, val in enumerate(obs_y):
        if val == 0:
            try:
                obs_y[i] = obs_y[i-1]
            except:
                pass       
    
    obs_y = np.array(obs_y)
    
    try:
        z = np.polyfit(obs_x, obs_y, 4)
        p = np.poly1d(z)
        pred_y = p(obs_x)
            
        forecasted_x = np.array(list(range(max(obs_x) + 7)))
        forecasted_y = p(forecasted_x)
    except:
        pass
    
    return forecasted_y, forecasted_x, pred_y



def get_best(obs_x, obs_y, model, df_sub):
    
    obs_x = np.array(obs_x)
    for i, val in enumerate(obs_y):
        if val == 0:
            try:
                obs_y[i] = obs_y[i-1]
            except:
                pass     
            
    obs_y = np.array(obs_y)
    
    best_r2 = 0
    best_model = str()
    best_loc = str()
    best_pred_y = []
    best_forecasted_x = []
    best_forecasted_y = []
    
    bench_x, bench_y = [], []
    
    locs = list(set(df_sub['Province/State']))
    for loc in locs:
        df_benchmark = df_sub[df_sub['Province/State'] == loc]
        bench = df_benchmark.iloc[0,6:].values
        
        bench_y = []
        for i, val in enumerate(bench):
            if len(bench_y) > 0 or val > 0:
                bench_y.append(val)

        bench_x = list(range(len(bench_y)))
        if len(bench_x) <= len(obs_x) or sum(bench_y) <= sum(obs_y): 
            continue
        
        z = np.polyfit(bench_x, bench_y, 3)
        p = np.poly1d(z)
        pred_y = p(obs_x)
            
        forecasted_x = np.array(list(range(len(obs_x) + 7)))
        forecasted_y = p(forecasted_x)
        
        obs_pred_r2 = obs_pred_rsquare(obs_y, pred_y)
        #print(obs_pred_r2)
        
        if obs_pred_r2 > best_r2:
            best_r2 = float(obs_pred_r2)
            best_loc = loc
            best_pred_y = pred_y
            best_forecasted_x = forecasted_x
            best_forecasted_y = forecasted_y
            
    if len(best_forecasted_x) == 0:
        print('Failed to find a Best Benchmark failed. Using 3rd degree polynomial.')
        best_forecasted_y, best_forecasted_x, best_pred_y = get_polynomial(obs_x, obs_y)
        best_loc = 'None'
        best_r2 = 0

    return best_r2, best_loc, best_pred_y, best_forecasted_x, best_forecasted_y



def fit_curve(obs_x, obs_y, model, df_sub):

    obs_x = list(range(len(obs_y)))
    obs_x = np.array(obs_x)
    obs_y = np.array(obs_y)
    
    best_loc = str()
    
    if model == 'Best Benchmark':
        obs_pred_r2, best_loc, pred_y, forecasted_x, forecasted_y = get_best(obs_x, obs_y, model, df_sub)
        if obs_pred_r2 == 0 and best_loc == 'None':
            obs_pred_r2 = obs_pred_rsquare(obs_y, pred_y)
            
    elif model == 'logistic':
        forecasted_y, forecasted_x, pred_y = get_logistic(obs_x, obs_y)
        obs_pred_r2 = obs_pred_rsquare(obs_y, pred_y)
    elif model == 'exponential':
        forecasted_y, forecasted_x, pred_y = get_exponential(obs_x, obs_y)
        obs_pred_r2 = obs_pred_rsquare(obs_y, pred_y)
    elif model == 'polynomial':
        forecasted_y, forecasted_x, pred_y = get_polynomial(obs_x, obs_y)
        obs_pred_r2 = obs_pred_rsquare(obs_y, pred_y)
        
    return obs_pred_r2, model, best_loc, obs_x, pred_y, forecasted_x, forecasted_y


In [5]:
class App_GetFits:
    
    def __init__(self, df):
        
        # model: 'logistic'; 'exponential'; 'polynomial';
        # query: Any location available within the dataframe
        # refer: Any location available within the dataframe
        
        self._df = df
        available_indicators2 = list(set(self._df['Province/State']))
        available_indicators2.sort()
        
        self._1_dropdown = self._create_dropdown(['logistic', 'exponential', 'polynomial', 'Best Benchmark'], 2, label = 'Choose model:')
        self._2_dropdown = self._create_dropdown(available_indicators2, 98, label = 'Choose location:')
        self._plot_container = widgets.Output()
        
        _app_container = widgets.VBox(
            [widgets.VBox([self._1_dropdown, self._2_dropdown]),
            self._plot_container], layout=widgets.Layout(align_items='flex-start', flex='0 0 auto'))
                
        # 'flex-start', 'flex-end', 'center', 'baseline', 'stretch', 'inherit', 'initial', 'unset'
        self.container = widgets.VBox([
            widgets.HBox([
                _app_container
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()
        
        
    @classmethod
    def from_url(cls):
        
        _URL = 'https://raw.githubusercontent.com/klocey/ResourceDemand/master/notebooks/COVID-CASES-DF.txt'
        df = pd.read_csv(_URL, sep='\t')  
        df = df.drop(df.columns[0], axis=1)
        
        return cls(df)
        
        
    def _create_dropdown(self, indicators, initial_index, label):
        dropdown = widgets.Dropdown(options=indicators, 
                                    layout={'width': 'initial'},
                                    style={'description_width': 'initial'},
                                    value=indicators[initial_index],
                                   description=label)
        dropdown.observe(self._on_change, names=['value'])
        return dropdown
    
    
    def _get_fit(self, model, loc):
        
        global obs_pred_r2_G
        global model_G
        global loc_G
        global obs_x_G
        global obs_y_G
        global pred_y_G
        global forecasted_x_G
        global forecasted_y_G
        global dates
        global fdates
        
        # model = 'logistic', 'exponential', 'polynomial', 'Best Benchmark'
        
        df_sub = self._df[self._df['type'] == 'Confirmed']
        
        yi = list(df_sub)
        DATES = yi[6:]
        #print(DATES)
        #sys.exit()
        
        df_loc = df_sub[df_sub['Province/State'] == loc]
        focal = df_loc.iloc[0,6:].values
        
        y = []
        dates = []
        for i, val in enumerate(focal):
            if len(y) > 0 or val > 0:
                y.append(val)
                dates.append(DATES[i])

        x = list(range(len(y)))
        obs_pred_r2_G, model_G, loc_G, obs_x_G, pred_y_G, forecasted_x_G, forecasted_y_G = fit_curve(x, y, model, df_sub)
        obs_y_G = np.array(list(y))
        
        if obs_pred_r2_G < 0:
            obs_pred_r2_G = 0.0
        
        fig = plt.figure(figsize=(11,10))
        
        ax = plt.subplot2grid((2, 2), (0,0), colspan=1, rowspan=1)
        #ax = plt.gca()

        y = np.array(y)
        y[y < 0] = 0
        pred_y_G = np.array(pred_y_G)
        pred_y_G[pred_y_G < 0] = 0

        forecasted_y_G = np.array(forecasted_y_G)
        forecasted_y_G[forecasted_y_G < 0] = 0
        forecast_vals = np.copy(forecasted_y_G)
        
        numdays = len(forecasted_x_G)
        latest_date = pd.to_datetime(dates[-1])
        first_date = pd.to_datetime(dates[0])

        future_date = latest_date + datetime.timedelta(days = 6)
        fdates = pd.date_range(start=first_date, end=future_date)
        
        plt.plot(fdates, forecasted_y_G, c='Crimson', linewidth=3, label='Forecasted')

        latest_date = dates[-1]
        first_date = dates[0]
        dates = pd.date_range(start=first_date, end=latest_date)
        
        lab1 = r'$r^{2}$' + ' = ' + str(np.round(obs_pred_r2_G, 2))
        plt.scatter(dates, y, c='0.2', s=100, alpha=0.3, linewidths=0.1)
        plt.plot(dates, pred_y_G, c='steelblue', linewidth=3, label='Predicted ' + lab1)

        new_cases = []
        for i, val in enumerate(forecast_vals):
            if i > 0:
                forecast_vals[i] = forecast_vals[i] - forecast_vals[i-1]
                new_cases.append(forecast_vals[i])
                
        leg = ax.legend(handlelength=0, handletextpad=0, fancybox=False,
                        loc='best', frameon=False, fontsize=14)

        for line,text in zip(leg.get_lines(), leg.get_texts()):
            text.set_color(line.get_color())

        for item in leg.legendHandles: 
            item.set_visible(False)

        plt.xticks(rotation=35, ha='center')
        plt.xlabel('Date', fontsize=14, fontweight='bold')
        plt.ylabel('Confirmed cases', fontsize=14, fontweight='bold')
        
        
        ax = plt.subplot2grid((2, 2), (0,1), colspan=1, rowspan=1)
        ax.axis('off')
        ax.axis('tight')
        
        if loc == 'Cook County, IL':  
            loc = 'Cook County'
            loc2 = 'Rush'

        
        if len(loc) > 12:
            loc = loc[:12]
            loc = loc + '...'
        
        if loc == 'Cook County':
            col_labels = ['Cumulative', 'New', 'New (Rush)']
        else:
            col_labels = ['Cumulative', 'New']
        
        row_labels = fdates.strftime('%m-%d')
        row_labels = row_labels.tolist()
        row_labels = row_labels[-8:]
        
        new_cases = np.round(new_cases, 0).astype('int')
        new_cases = new_cases[-8:]
    
        sub_f = forecasted_y_G[-8:]
        
        table_vals = []
        for i in range(len(row_labels)):
            
            val = new_cases[i]
            if loc == 'Cook County':
                cell = [int(np.round(sub_f[i])), val, int(np.round(val*0.25))]
            else:
                cell = [int(np.round(sub_f[i])), val]
                
            table_vals.append(cell)
        
        ncol = 2
        if loc == 'Cook County':
            ncol = 3
        the_table = plt.table(cellText=table_vals,
                      colWidths=[0.31] * ncol,
                      rowLabels=row_labels,
                      colLabels=col_labels,
                      cellLoc='center',
                      loc='lower center')
        the_table.auto_set_font_size(False)
        the_table.set_fontsize(12)
        the_table.scale(1, 2)
        #plt.text(-0.2, 1.05, 'Forecasted cases for '+ loc, fontsize = 16, fontweight = 'bold')
        
        plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=None)
        
    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):
        
        model = self._1_dropdown.value
        focal_loc = self._2_dropdown.value
        
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._get_fit(model, focal_loc)
            plt.show()
        
    

In [6]:
%config InlineBackend.figure_format = 'svg'

app1 = App_GetFits.from_url()
#app2 = App_CasesTable.from_url()
#app3 = App_GetFits.from_url()

# Put the views into a layout grid
grid = widgets.GridspecLayout(1, 1, layout=widgets.Layout(justify_content='flex-start'))
####  'flex-start', 'flex-end', 'center', 'space-between', 'space-around', 'inherit', 'initial', 'unset'
grid[0, 0] = app1.container
#grid[0, 1] = app2.container
#grid[0, 2] = app3.container

app_contents = [grid]
app = widgets.VBox(app_contents)

display(app)